### **利用gdal开源库进行栅格文件处理**
#### 1. 投影转换
#### 2. 重采样


In [2]:
from osgeo import gdal
from osgeo import osr
import numpy as np


In [3]:
## 读入栅格数据，包括影像信息和地理信息
dset = gdal.Open('data/Section_5/rsimg_chenggong_wgs84.tif')
proj = dset.GetProjection()    ### 获取影像投影
print('projection:', proj)
geo_trans = dset.GetGeoTransform()
print(geo_trans)
x_min = geo_trans[0]
x_max = geo_trans[0]+geo_trans[1]*dset.RasterXSize
y_max = geo_trans[3]
y_min = geo_trans[3]+geo_trans[5]*dset.RasterYSize
print(x_min, x_max, y_min, y_max)


projection: GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
(102.703814432, 0.00018789276986649867, 0.0, 24.993242602, 0.0, -0.00018789276977491845)
102.703814432 102.999369759 24.701069345 24.993242602


#### 1. 投影转换

In [4]:
### 计算utm投影带 (利用x_mean计算utm投影带，获得投影带号为48).
x_mean = (x_min + x_max)/2   
print(x_mean)


102.8515920955


In [5]:
### 左上角utm坐标计算, 及utm投影下影像范围
wgs84_epsg = 4326
utm48_epsg = 32648
wgs84 = osr.SpatialReference(); wgs84.ImportFromEPSG(wgs84_epsg)
utm48 = osr.SpatialReference(); utm48.ImportFromEPSG(utm48_epsg)
transform = osr.CoordinateTransformation(wgs84, utm48)
print(geo_trans[0], geo_trans[3])
(x_min_utm, y_max_utm, z) = transform.TransformPoint(geo_trans[3], geo_trans[0], 0)  ## 输入参数为(纬度，经度）,输出参数为（x，y），注意顺序
(x_max_utm, y_min_utm, z) = transform.TransformPoint(y_min, x_max, 0)  ## 输入参数为(纬度，经度, 输出参数为（x，y），注意顺序
print('Extent under the utm:', x_min_utm, x_max_utm, y_min_utm, y_max_utm)



102.703814432 24.993242602
Extent under the utm: 268239.7411192706 297604.322419359 2733325.2755200677 2766162.361361425


In [6]:
### 像元尺寸（分辨率）计算
width_pixel = (x_max_utm-x_min_utm)/dset.RasterXSize
height_pixel = (y_max_utm-y_min_utm)/dset.RasterYSize
print(width_pixel, height_pixel)


18.66788385256731 21.117097004088258


In [8]:
### 定义新的转换参数和投影
geotrans_new = (268239.7411192706, 18.66788385256731, 0.0, 2766162.361361425, 0, -21.117097004088258)
proj_new = utm48.ExportToWkt()


In [14]:
driver = gdal.GetDriverByName('GTiff')
dset_utm = driver.Create('data/Section_5/rsimg_chenggong_utm48.tif', xsize=dset.RasterXSize, \
                                ysize=dset.RasterYSize, bands=dset.RasterCount, eType=gdal.GDT_Int16)
dset_utm.SetGeoTransform(geotrans_new)
dset_utm.SetProjection(proj_new)

## 影像重投影
reproj = gdal.ReprojectImage(dset, dset_utm, \
                  wgs84.ExportToWkt(), utm48.ExportToWkt(), gdal.GRA_NearestNeighbour)
dset_utm = None  ## 关闭文件


### 重采样

In [12]:
res=50  ## 定义分辨率
### 1.计算影像尺寸
x_size = int((x_max_utm - x_min_utm)/res)  ## x方向尺寸
y_size = int((y_max_utm - y_min_utm)/res)   ## y方向尺寸
print(x_size, y_size)
### 2.更新影像分辨率
res_x = (x_max_utm - x_min_utm)/x_size  ##  新的x方向分辨率（像元宽）
res_y = (y_max_utm - y_min_utm)/y_size  ##  新的y方向分辨率（像元高）
print(res_x, res_y)
### 3. 定义地理转换参数
geotrans_resample = (268239.7411192706, 50.017679524931665, 0, 2766162.361361425, 0, -50.00278992423364)


587 656
50.02484037493761 50.056533294751894


In [15]:
driver = gdal.GetDriverByName('GTiff')
dset_resample = driver.Create('data/Section_5/rsimg_chenggong_utm48_resample.tif', xsize=x_size, \
                                ysize=y_size, bands=dset.RasterCount, eType=gdal.GDT_Int16)
dset_resample.SetGeoTransform(geotrans_resample)
dset_resample.SetProjection(proj_new)
## 影像重投影
reproj = gdal.ReprojectImage(dset, dset_resample, \
                  wgs84.ExportToWkt(), utm48.ExportToWkt(), gdal.GRA_NearestNeighbour)
dset_resample = None  ## 关闭文件


### 快捷方式
主要函数：gdal.Warp()/gdalwarp   
参考：   
1.https://gdal.org/api/python/osgeo.gdal.html.   
2.https://gdal.org/programs/gdalwarp.html

In [19]:
path_dset = 'data/Section_5/rsimg_chenggong_wgs84.tif'
path_dset_reproj = 'data/Section_5/rsimg_chenggong_reproj2utm48_by_warp.tif'
path_dset_resam = 'data/Section_5/rsimg_chenggong_utm48_resam_by_warp.tif'
### 重投影
warp_reproj = gdal.Warp(destNameOrDestDS=path_dset_reproj, srcDSOrSrcDSTab=path_dset, dstSRS='EPSG:32648')  ### 利用gdal.Warp()进行投影转换
warp_reproj = None   ### !!关闭文件


In [20]:
### 重采样
warp_resam = gdal.Warp(destNameOrDestDS=path_dset_resam, srcDSOrSrcDSTab=path_dset_reproj, xRes=50, yRes=50)  ### 利用gdal.Warp()进行投影转换
warp_resam = None   ### !!关闭文件

